<a href="https://colab.research.google.com/github/onurkasap/Computer-Vision/blob/main/cars_object_detections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics -q
!pip install opencv-python-headless -q

# Library

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
from google.colab import files
from IPython.display import HTML
from base64 import b64encode
import os

In [ ]:
print("Your files upload:")
#uploaded = files.upload()
video_path = 'cars1.mp4'

#video_path = list(uploaded.keys())[0]

Your files upload:


In [ ]:
if not os.path.exists(video_path):
    print(f"ERROR: '{video_path}' NOT FOUND!")
    print("Available files:")
    print(os.listdir())
else:
    print(f"Video found: {video_path}")


Video found: cars1.mp4


In [ ]:
print(f"✅ Installed video: {video_path}\n")

✅ Installed video: cars1.mp4



In [ ]:
print("Yolo install")
model = YOLO('yolov8n.pt')

Yolo install


# Vehicle Counting System

In [ ]:
class VehicleCounter:
  def __init__(self, line_position=0.5):
    self.going_count = 0
    self.coming_count = 0
    self.tracked_objects = {}
    self.line_position = line_position
    self.counted_ids = set()

  def update(self, detections, frame_height):
    line_y = int(frame_height * self.line_position)

    for detection in detections:
      x1, y1, x2, y2 = detection['bbox']
      object_id = detection['id']

      # if center point counting
      #center_y = (y1 + y2) / 2

      bottom_y = y2
      top_y = y1

      if object_id in self.tracked_objects:  #Önce önceki konum var mı?
        prev_bottom, prev_top = self.tracked_objects[object_id]

        if object_id not in self.counted_ids:
          if prev_bottom < line_y and bottom_y >= line_y:
            self.going_count += 1
            self.counted_ids.add(object_id)
            print(f" GOING: Total: {self.going_count}")

          elif prev_top > line_y and top_y <= line_y:
            self.coming_count += 1
            self.counted_ids.add(object_id)
            print(f" COMING: Total: {self.coming_count}")

      self.tracked_objects[object_id]= (bottom_y, top_y)

    return line_y


In [ ]:
def draw_info(frame, counter, line_y):
  h, w = frame.shape[:2]

  # Count line
  cv2.line(frame, (0, line_y), (w, line_y), (0, 255, 255), 3)
  cv2.putText(frame, 'COUNT LINE', (10, line_y - 10),
              cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255),2)

  cv2.rectangle(frame, (w - 350, 10), (w - 10, 120), (0, 0, 0), -1)
  cv2.rectangle(frame, (w - 350, 10), (w - 10, 120), (255, 255, 255), 2)

  # GOING - GREEN
  cv2.putText(frame, f"GOING: {counter.going_count}", (w - 330, 50),
              cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

  # COMING - RED
  cv2.putText(frame, f"COMING: {counter.coming_count}", (w - 330, 95),
              cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 100, 255), 3)


  return frame


# VIDEO PROCESSING

In [ ]:
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

output_path = 'output_vehicle_count.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Create Counter
counter = VehicleCounter(line_position=0.5)
frame_count = 0

while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    break

  frame_count += 1

  results = model.track(frame, persist= True, verbose=False,
                        classes=[2, 3, 5, 7])
  detections = []
  if results[0].boxes is not None and results[0].boxes.id is not None:
    boxes = results[0].boxes.xyxy.cpu().numpy()
    ids = results[0].boxes.id.cpu().numpy().astype(int)
    classes = results[0].boxes.cls.cpu().numpy().astype(int)

    for box, obj_id, cls in zip(boxes, ids, classes):
      x1, y1, x2, y2 = box
      detections.append({
          'bbox': (int(x1), int(y1), int(x2), int(y2)),
          'id': obj_id,
          'class': cls
      })

      #Bounding Box
      cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
      cv2.putText(frame, f"ID:{obj_id}", (int(x1), int(y1)-10),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)


  # Update Counter
  line_y = counter.update(detections, height)

  # Information Line
  frame = draw_info(frame, counter, line_y)

  # Save the frame
  out.write(frame)

  # View Forward
  if frame_count % 30 == 0:
    progress = (frame_count / total_frames) * 100
    print(f" Processing: {progress:.1f}% - Goes: {counter.going_count}, Comes: {counter.coming_count}")

cap.release()
out.release()

 GOING: Total: 1
 GOING: Total: 2
 COMING: Total: 1
 GOING: Total: 3
 Processing: 1.7% - Goes: 3, Comes: 1
 COMING: Total: 2
 COMING: Total: 3
 GOING: Total: 4
 Processing: 3.3% - Goes: 4, Comes: 3
 COMING: Total: 4
 GOING: Total: 5
 COMING: Total: 5
 GOING: Total: 6
 Processing: 5.0% - Goes: 6, Comes: 5
 GOING: Total: 7
 COMING: Total: 6
 GOING: Total: 8
 COMING: Total: 7
 COMING: Total: 8
 Processing: 6.7% - Goes: 8, Comes: 8
 COMING: Total: 9
 COMING: Total: 10
 COMING: Total: 11
 Processing: 8.3% - Goes: 8, Comes: 11
 COMING: Total: 12
 Processing: 10.0% - Goes: 8, Comes: 12
 GOING: Total: 9
 COMING: Total: 13
 COMING: Total: 14
 GOING: Total: 10
 Processing: 11.7% - Goes: 10, Comes: 14
 COMING: Total: 15
 Processing: 13.3% - Goes: 10, Comes: 15
 COMING: Total: 16
 COMING: Total: 17
 COMING: Total: 18
 Processing: 15.0% - Goes: 10, Comes: 18
 GOING: Total: 11
 COMING: Total: 19
 COMING: Total: 20
 GOING: Total: 12
 COMING: Total: 21
 Processing: 16.7% - Goes: 12, Comes: 21
 GOING: 

In [ ]:
print("\nCompleted video processing!")
print(f"RESULTS:")
print(f"  Going Cars: {counter.going_count}")
print(f"  Coming Cars: {counter.coming_count}")
print(f"  TOTAL: {counter.going_count + counter.coming_count}")


Completed video processing!
RESULTS:
  Going Cars: 98
  Coming Cars: 108
  TOTAL: 206


In [ ]:
print("\n📺 Prepare the video...")

# Download the video
from google.colab import files
files.download(output_path)


# Playing Colab
mp4 = open(output_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<div style="text-align: center;">
  <h2>✅ Process Video</h2>
  <video width="800" controls>
    <source src="{data_url}" type="video/mp4">
  </video>
  <p><b>Going:</b> {counter.going_count} | <b>Coming:</b> {counter.coming_count} | <b>Toplam:</b> {counter.going_count + counter.coming_count}</p>
</div>
""")



📺 Prepare video...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>